In [14]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory
import numpy as np
from sklearn.model_selection import train_test_split

print("Using TensorFlow version:", tf.__version__)

# Pour reproduire un comportement stable (optionnel)
tf.random.set_seed(42)

Using TensorFlow version: 2.18.0


## 1. Chargement du dataset

In [ ]:
# # Chemin vers le dossier qui contient les deux sous-dossiers : 'good' et 'defect'
# DATA_DIR = "screw_dataset"  # À adapter

# BATCH_SIZE = 32
# IMG_SIZE = (224, 224)

# # Charger toutes les images d'un seul tenant
# dataset = image_dataset_from_directory(
#     DATA_DIR,
#     labels='inferred',
#     label_mode='categorical',  # Pour un problème multi-classes, ici on suppose 2 classes (good, defect)
#     batch_size=BATCH_SIZE,
#     image_size=IMG_SIZE,
#     shuffle=True  # Mélange les données
# )

# # Afficher les classes détectées
# class_names = dataset.class_names
# print("Classes détectées :", class_names)

# # Optionnel : Normalisation et mise en cache
# AUTOTUNE = tf.data.AUTOTUNE
# def preprocess(image, label):
#     image = tf.cast(image, tf.float32) / 255.0  # Normalisation simple
#     return image, label

# dataset = dataset.map(preprocess).cache().prefetch(buffer_size=AUTOTUNE)

Found 1152 files belonging to 2 classes.
Classes détectées : ['bad', 'good']


In [ ]:
# Chemins et paramètres
DATA_DIR = "screw_dataset"
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
NUM_SAMPLES_TO_SHOW = 10  # Nombre d'exemples à afficher
TRAIN_RATIO = 0.7
VAL_RATIO = 0.15
TEST_RATIO = 0.15

print("Chargement des images depuis les dossiers...")

# Charger le dataset
dataset = image_dataset_from_directory(
    DATA_DIR,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=False
)

# Afficher les classes détectées
class_names = dataset.class_names
print("Classes détectées :", class_names)

# Convertir le dataset en tableaux numpy
all_images = []
all_labels = []

for images, labels in dataset:
    all_images.append(images.numpy())
    all_labels.append(labels.numpy())

# Concaténer tous les batches
X = np.concatenate(all_images, axis=0)
y = np.concatenate(all_labels, axis=0)

# Normaliser les valeurs des pixels
X = X / 255.0

print(f"Nombre total d'images: {X.shape[0]}")

# Distribution originale des classes
class_counts = np.sum(y, axis=0)
total_images = np.sum(class_counts)
print("\n=== DISTRIBUTION DES CLASSES DANS LES DONNÉES ORIGINALES ===")
for i, class_name in enumerate(class_names):
    count = int(class_counts[i])
    percentage = (count / total_images) * 100
    print(f"Classe '{class_name}' (indice {i}): {count} images ({percentage:.1f}%)")
print("=" * 60)

# Créer des indices pour shuffle et garder la correspondance image/label
indices = np.arange(X.shape[0])
np.random.seed(42)  # Pour la reproductibilité
np.random.shuffle(indices)

# Mélanger les données
X_shuffled = X[indices]
y_shuffled = y[indices]

# Afficher les premières lignes du tableau shufflé dans le format simplifié
print("\n=== TABLEAU SHUFFLÉ (FORMAT SIMPLIFIÉ) ===")
for i in range(NUM_SAMPLES_TO_SHOW):
    label_index = np.argmax(y_shuffled[i])
    label_name = class_names[label_index]
    print(f"[{i}] | classe={label_index} ({label_name}) | one-hot={y_shuffled[i]}")
print("=" * 60)

# Division en ensembles train, validation et test
print("\nDivision des données en ensembles d'entraînement, validation et test...")

# Diviser en train et temp (validation + test)
X_train, X_temp, y_train, y_temp = train_test_split(
    X_shuffled, 
    y_shuffled,
    train_size=TRAIN_RATIO,
    random_state=42,
    stratify=np.argmax(y_shuffled, axis=1)  # Assurer l'équilibre des classes
)

# Diviser temp en validation et test
val_test_ratio = VAL_RATIO / (VAL_RATIO + TEST_RATIO)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, 
    y_temp,
    train_size=val_test_ratio,
    random_state=42,
    stratify=np.argmax(y_temp, axis=1)  # Assurer l'équilibre des classes
)

print(f"Ensemble d'entraînement: {X_train.shape[0]} images")
print(f"Ensemble de validation: {X_val.shape[0]} images")
print(f"Ensemble de test: {X_test.shape[0]} images")

# Afficher quelques exemples de l'ensemble d'entraînement
print("\n=== EXEMPLES DE L'ENSEMBLE D'ENTRAÎNEMENT (FORMAT SIMPLIFIÉ) ===")
for i in range(NUM_SAMPLES_TO_SHOW):
    label_index = np.argmax(y_train[i])
    label_name = class_names[label_index]
    print(f"[{i}] | classe={label_index} ({label_name}) | one-hot={y_train[i]}")
print("=" * 60)

# Créer les datasets TensorFlow
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# Configurer les datasets pour l'entraînement
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.shuffle(buffer_size=X_train.shape[0]).batch(BATCH_SIZE).prefetch(AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)

# Vérifier la distribution des classes dans chaque ensemble
train_class_counts = np.sum(y_train, axis=0)
val_class_counts = np.sum(y_val, axis=0)
test_class_counts = np.sum(y_test, axis=0)

print("\n=== STATISTIQUES DES ENSEMBLES ===")
print("Distribution des classes dans chaque ensemble:")
for i, class_name in enumerate(class_names):
    print(f"Classe '{class_name}' (indice {i}):")
    print(f"  - Train: {int(train_class_counts[i])} images ({train_class_counts[i]/np.sum(train_class_counts)*100:.1f}%)")
    print(f"  - Validation: {int(val_class_counts[i])} images ({val_class_counts[i]/np.sum(val_class_counts)*100:.1f}%)")
    print(f"  - Test: {int(test_class_counts[i])} images ({test_class_counts[i]/np.sum(test_class_counts)*100:.1f}%)")
print("=" * 60)

print("\nLes datasets sont prêts à être utilisés pour l'entraînement!")

Chargement des images depuis les dossiers...
Found 1152 files belonging to 2 classes.
Classes détectées : ['bad', 'good']
Nombre total d'images: 1152

=== DISTRIBUTION DES CLASSES DANS LES DONNÉES ORIGINALES ===
Classe 'bad' (indice 0): 285 images (24.7%)
Classe 'good' (indice 1): 867 images (75.3%)

=== TABLEAU SHUFFLÉ (FORMAT SIMPLIFIÉ) ===
[0] | classe=0 (bad) | one-hot=[1. 0.]
[1] | classe=1 (good) | one-hot=[0. 1.]
[2] | classe=1 (good) | one-hot=[0. 1.]
[3] | classe=0 (bad) | one-hot=[1. 0.]
[4] | classe=1 (good) | one-hot=[0. 1.]
[5] | classe=0 (bad) | one-hot=[1. 0.]
[6] | classe=1 (good) | one-hot=[0. 1.]
[7] | classe=1 (good) | one-hot=[0. 1.]
[8] | classe=1 (good) | one-hot=[0. 1.]
[9] | classe=1 (good) | one-hot=[0. 1.]

Division des données en ensembles d'entraînement, validation et test...
Ensemble d'entraînement: 806 images
Ensemble de validation: 173 images
Ensemble de test: 173 images

=== EXEMPLES DE L'ENSEMBLE D'ENTRAÎNEMENT (FORMAT SIMPLIFIÉ) ===
[0] | classe=1 (goo

## 2. Définition d'un modèle simple
On définit un modèle CNN

In [4]:
num_classes = len(class_names)

model = models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['accuracy']
)

model.summary()

/Users/emmacoco/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 93312)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     5,972,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,977,250 (22.80 MB)

 Trainable params: 5,977,250 (22.80 MB)

 Non-trainable params: 0 (0.00 B)

## 3. Entraînement du modèle

In [ ]:
EPOCHS = 1000  # Nombre d'époques d'entraînement

history = model.fit(
    dataset,                 # Entraînement ET test sur le même dataset
    epochs=EPOCHS,
    verbose=1
)

## 4. Évaluation sur le même dataset

In [ ]:
loss, accuracy = model.evaluate(dataset, verbose=0)
print(f"Loss sur le dataset complet : {loss:.4f}")
print(f"Accuracy sur le dataset complet : {accuracy:.4f}")